<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# Test Single Module External Reference

### Description:

The purpose of this script is to test external reference clock.
In an instrument with one or more module.

### Equipment

- 1x Proteues P948x device or or P9488D.(DUT) 
- Scope.
- Signal-Generator.

### set up:

* connect channel 1 of Proteus to the channel 1 of the scope.
* Start a Proteus Lan Service - either use Tabor Instrumentation service or run TEPAdminTestServer.exe. 
* Set the signal genertor clock to 10MHz.
* Connect the signal genertor to the to ref-clock in the proteus and to channel 2 of the scope.
  
 
 (TODO - Add automatic Scope Measurement - by connect the Scope by usb to the computer.)

Run the test 


### Proteus version: 
AMANDA

In [3]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [4]:
import os
import gc
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

### Before we start

This example is designed to work both when there is a LAN based communication-service (usually on benchtop/desktop)<br>
And when there's no communication-service, and the user's application has access to hardware-modules in the PXI chassis.

In the first case, when there's a LAN communication, this example uses the class `TEVisaInst` from `tevisainst`.<br>

In the second case, this example uses the classes `TEProteusAdmin` and `TEProteusInst` from `teproteus`.

In [5]:
# Connect to instrument

inst = None
admin = None

conn_msg_str = ''

def disconnect():
    global inst
    global admin
    if inst is not None:
        try:
            inst.close_instrument()            
        except:
            pass
        inst = None
    if admin is not None:
        try:
            admin.close_inst_admin()
        except:
            pass
        admin = None
    gc.collect()

def connect_to_pxi_slot(slot_id):
    global inst
    global admin
    try:
        disconnect()        
        admin = TEProteusAdmin()
        admin.open_inst_admin()        
        inst = admin.open_instrument(slot_id, reset_hot_flag=True)
    except:
        pass

def connect_to_lan_server(ip_address):
    global inst
    try:
        disconnect()
        inst = TEVisaInst(ip_address, port=5025)
    except:
        pass

def connect(ipaddr_or_slotid):
    try:
        disconnect()
        if isinstance(ipaddr_or_slotid, str) and '.' in ipaddr_or_slotid:
            connect_to_lan_server(ipaddr_or_slotid)
        else:
            connect_to_pxi_slot(ipaddr_or_slotid)
    except:
        pass

addr_textbox = widgets.Text(
    value='',
    placeholder='IP Address or Slot Id',
    description='Address:',
    disabled=False
)

output = widgets.Output()

def print_conn_message(msg_str):
    global output
    global conn_msg_str
    with output:
        if len(conn_msg_str) > 0:
            print('\r', end='')
            print(' ' * len(conn_msg_str), end='')
            print('\r', end='')
        conn_msg_str = str(msg_str)
        if len(conn_msg_str) > 0:
            print(conn_msg_str, end='')

def on_button_connect_clicked(b):
    global inst
    global addr_textbox
    print_conn_message('')
    disconnect()
    if addr_textbox is not None:
        conn_str = addr_textbox.value
        if conn_str:
            print_conn_message('Connecting to {0} ..'.format(conn_str))
            connect(conn_str)
            print_conn_message('')
    if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        if idn_str:
            print_conn_message('Connected to: ' + idn_str.strip())
    
def on_button_disconnect_clicked(b):
    print_conn_message('')
    disconnect()

btn_connect = widgets.Button(description="Connect")
btn_disconnect = widgets.Button(description="Disconnect")

btn_connect.on_click(on_button_connect_clicked)
btn_disconnect.on_click(on_button_disconnect_clicked)

display(addr_textbox, btn_connect, btn_disconnect, output)

Text(value='', description='Address:', placeholder='IP Address or Slot Id')

Button(description='Connect', style=ButtonStyle())

Button(description='Disconnect', style=ButtonStyle())

Output()

In [6]:
connect("192.168.0.117")
if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        if idn_str:
            print_conn_message('Connected to: ' + idn_str.strip())
    

In [7]:
# Set the default paranoia-level (0, 1 or 2)
inst.default_paranoia_level = 2 # good for debugging

# Clear error-list and reset the instrument
inst.send_scpi_cmd('*CLS; *RST')

0

In [8]:
# Get the model name
model = inst.send_scpi_query(":SYST:iNF:MODel?")
print("Model: " + model)

# Infer the natural DAC waveform format
if 'P908' in model:
    dac_mode = 8
else:
    dac_mode = 16
    


print("DAC waveform format: {0} bits-per-point".format(dac_mode))

max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0
print('Max DAC wave-point level: {0}'.format(max_dac))
print('Half DAC wave-point level: {0}'.format(half_dac))

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))


Model: P9484M
DAC waveform format: 16 bits-per-point
Max DAC wave-point level: 65535
Half DAC wave-point level: 32767.5
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes


## Writing waveform data to segment

The command `:TRACE:DATA [<offset-in-bytes>] #<binary-header><binary-data>`  
writes the given block of binary data to the specified offset in the selected segment of the selected channel.

#### The header of the binary-data block
The SCPI protocol uses text-based messages, and the new-line character denotes end of a SCPI statement.<br>
In case of binary-data the sender must send a header that defines the size of the binary block before the data  
because the new-line character can be a part of the data.<br>

The header is composed of `'#'` character followed by a sequence of between 2 and 10 decimal digits, where the<br>
first digits denotes how many digits follows it (between 1 and 9) and the rest of the digits denote the size of the<br>
binary-block in bytes. For example, `#3256` denotes block of 256 bytes, and `#41024` denotes block of 1024 bytes.

### Simple example

Download waves to several segments and play different segment in each channel.  


In [9]:
# Build waveforms

seglen = 8000
num_cycles = [32]*num_channels #[32,32,32,32]
SCLK=2.5e9

waves = [ None for _ in range(num_channels)]

for ii in range(num_channels):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(
        start=0, stop=seglen, num=seglen, endpoint=False)
    y = x % cyclelen > (cyclelen/2)
    y = y.astype(np.uint16) *max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        waves[ii] = y.astype(np.uint16)
    else:
        waves[ii] = y.astype(np.uint8)
    


In [10]:
# Download segments
for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ichan % 2 + 1
    wav = waves[ichan]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
    
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)
    
    # Define segment
    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    inst.send_scpi_cmd(cmd)
    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)
        
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
        
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')
    
    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

print()

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)



Download wave to segment 1 of channel 1
0, no error
Download wave to segment 2 of channel 2
0, no error
Download wave to segment 1 of channel 3
0, no error
Download wave to segment 2 of channel 4
0, no error

0, no error


In [11]:
del waves
del wav
dac_mode = 16
SCLK=2.5e9
inst.send_scpi_cmd(':SOUR:FREQ {}'.format(SCLK))
resp = inst.send_scpi_query(':SOUR:FREQ?')
print(resp)
print(dac_mode)
    

2500000000.000
16


In [12]:
inst.send_scpi_cmd(':ROSC:SOUR EXT')
inst.send_scpi_cmd(':ROSC:FREQ 10M')

0

### Verify Test \#1 Result

Check Scope signals on channel 1 & 2 . Verify that the signals are synchronized, i.e not stable in the scope and not 'drifting' on in relation to the other.
In case DUT-P9488D - Connect Channel 5 of Proteus to channel 1 of Scope and verify that the signals are synchronized as well.

In [13]:
inst.send_scpi_cmd(':ROSC:SOUR INT')
inst.send_scpi_cmd(':ROSC:FREQ 10M')

0

### Verify Test \#2 Result

Check Scope signals on channel 1 & 2 . Verify that the signals are not synchronized.
In case DUT-P9488D - Connect Channel 5 of Proteus to channel 1 of Scope and verify that the signals are synchronized as well.

In [14]:
# Disconnect
print_conn_message('')
disconnect()